In [2]:
import sys
import os
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from newsrec_utils import prepare_hparams

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

os.environ["CUDA_VISIBLE_DEVICES"]="0"

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

System version: 3.6.11 | packaged by conda-forge | (default, Nov 27 2020, 18:57:37) 
[GCC 9.3.0]
Tensorflow version: 1.15.4
Num GPUs Available:  1


In [3]:
epochs = 8
seed = 42
batch_size = 32

MIND_type = 'small'

In [7]:
data_path = 'dataset/MIND'+MIND_type
train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'train_behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding_all.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict_all.pkl")
vertDict_file = os.path.join(data_path, "utils", "vert_dict.pkl")
subvertDict_file = os.path.join(data_path, "utils", "subvert_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'DRUIC.yaml')

valid_news_file = os.path.join(data_path, 'train', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'train', r'val_behaviors.tsv')
test_news_file = os.path.join(data_path, 'valid', r'news.tsv')
test_behaviors_file = os.path.join(data_path, 'valid', r'test_behaviors.tsv')

In [5]:
from mind_all_iterator import MINDAllIterator
from DRUIC import DRUIC
import json
from datetime import datetime

In [9]:
hparams = prepare_hparams(
    yaml_file, 
    wordEmb_file=wordEmb_file,
    wordDict_file=wordDict_file, 
    userDict_file=userDict_file,
    vertDict_file=vertDict_file, 
    subvertDict_file=subvertDict_file,
    batch_size=batch_size,
    epochs=epochs,
    show_step=10,
    support_quick_scoring=False
            )  

In [10]:
iterator = MINDAllIterator
model = DRUIC(hparams, iterator,seed=seed)


finish initial Iterator
finish initial Iterator
finish initial Iterator
start _build_graph...


In [ ]:
model.fit(train_news_file, train_behaviors_file,None, None)
res_syn = model.run_eval(test_news_file, test_behaviors_file,'test')